In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This is the smoker status dataset. Each row contains a person's health information and whether they smoke or not.

In [3]:
#data_path = '/content/drive/MyDrive/Colab Notebooks/Project 3/data/'
data_path = '/content/drive/MyDrive/Data Science/Project 3/Project 3/data/'
df = pd.read_csv(data_path + 'smoking.csv')
df.head()

,ID,gender,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),...,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,oral,dental caries,tartar,smoking
0,0,F,40,155,60,81.3,1.2,1.0,1.0,1.0,...,12.9,1.0,0.7,18.0,19.0,27.0,Y,0,Y,0
1,1,F,40,160,60,81.0,0.8,0.6,1.0,1.0,...,12.7,1.0,0.6,22.0,19.0,18.0,Y,0,Y,0
2,2,M,55,170,60,80.0,0.8,0.8,1.0,1.0,...,15.8,1.0,1.0,21.0,16.0,22.0,Y,0,N,1
3,3,M,40,165,70,88.0,1.5,1.5,1.0,1.0,...,14.7,1.0,1.0,19.0,26.0,18.0,Y,0,Y,0
4,4,F,40,155,60,86.0,1.0,1.0,1.0,1.0,...,12.5,1.0,0.6,16.0,14.0,22.0,Y,0,N,0


In [4]:
df.isna().sum()

ID                     0
gender                 0
age                    0
height(cm)             0
weight(kg)             0
waist(cm)              0
eyesight(left)         0
eyesight(right)        0
hearing(left)          0
hearing(right)         0
systolic               0
relaxation             0
fasting blood sugar    0
Cholesterol            0
triglyceride           0
HDL                    0
LDL                    0
hemoglobin             0
Urine protein          0
serum creatinine       0
AST                    0
ALT                    0
Gtp                    0
oral                   0
dental caries          0
tartar                 0
smoking                0
dtype: int64

In [5]:
df.shape

(55692, 27)

In [6]:
df.dtypes

ID                       int64
gender                  object
age                      int64
height(cm)               int64
weight(kg)               int64
waist(cm)              float64
eyesight(left)         float64
eyesight(right)        float64
hearing(left)          float64
hearing(right)         float64
systolic               float64
relaxation             float64
fasting blood sugar    float64
Cholesterol            float64
triglyceride           float64
HDL                    float64
LDL                    float64
hemoglobin             float64
Urine protein          float64
serum creatinine       float64
AST                    float64
ALT                    float64
Gtp                    float64
oral                    object
dental caries            int64
tartar                  object
smoking                  int64
dtype: object

In [7]:
df['smoking'].value_counts()

0    35237
1    20455
Name: smoking, dtype: int64

Data looks pretty clean - no NA values, and most columns are numeric. Here are some notes:

- Gender is 'M' or 'F'

- Hearing is listed as float but only has 1. or 2. for values. Systolic, relaxation, fasting blood sugar, cholesterol, triglyceride, HDL, LDL, AST, ALT, Gtp could also be ints

- urine protein should be categorical int, 1-6. order matters

- For 'oral' col, all entries have 'Y' - drop

- dental caries col is category, already has 0 and 1 for values

- tartar is 'Y' or 'N'

- smoking is 0 or 1 categorical


In [8]:
#dropping oral col
df.drop('oral', axis=1, inplace=True)

In [9]:
#changing gender and tartar cols to numeric categories
from sklearn.preprocessing import LabelEncoder
gender_le = LabelEncoder()
df['gender'] = gender_le.fit_transform(df['gender'])
tartar_le = LabelEncoder()
df['tartar'] = gender_le.fit_transform(df['tartar'])
#for gender, F is 0 and M is 1
#for tartar, is N 0 and Y is 1

In [10]:
gender_le.inverse_transform([0,1])

array(['N', 'Y'], dtype=object)

In [11]:
#changing float cols that all end in .0 to ints
float_to_int_cols = ['hearing(left)', 'hearing(right)', 'systolic', 'relaxation', 'fasting blood sugar', 'Cholesterol', 
                     'triglyceride', 'HDL', 'LDL', 'AST', 'ALT', 'Gtp', 'Urine protein']
for col in float_to_int_cols:
  df[col] = df[col].astype(int)
df.dtypes

ID                       int64
gender                   int64
age                      int64
height(cm)               int64
weight(kg)               int64
waist(cm)              float64
eyesight(left)         float64
eyesight(right)        float64
hearing(left)            int64
hearing(right)           int64
systolic                 int64
relaxation               int64
fasting blood sugar      int64
Cholesterol              int64
triglyceride             int64
HDL                      int64
LDL                      int64
hemoglobin             float64
Urine protein            int64
serum creatinine       float64
AST                      int64
ALT                      int64
Gtp                      int64
dental caries            int64
tartar                   int64
smoking                  int64
dtype: object

Renaming a couple of columns to be more consistent

In [12]:
#there doesn't seem to be a 'gtp' test, but there IS a ggt test
#originals:
# ['ID', 'gender', 'age', 'height(cm)', 'weight(kg)', 'waist(cm)',
#        'eyesight(left)', 'eyesight(right)', 'hearing(left)', 'hearing(right)',
#        'systolic', 'relaxation', 'fasting blood sugar', 'Cholesterol',
#        'triglyceride', 'HDL', 'LDL', 'hemoglobin', 'Urine protein',
#        'serum creatinine', 'AST', 'ALT', 'Gtp', 'dental caries', 'tartar',
#        'smoking']

df.columns = ['ID', 'gender', 'age', 'height(cm)', 'weight(kg)', 'waist(cm)',
       'eyesight(left)', 'eyesight(right)', 'hearing(left)', 'hearing(right)',
       'systolic', 'relaxation', 'fasting blood sugar', 'cholesterol',
       'triglyceride', 'HDL', 'LDL', 'hemoglobin', 'urine protein',
       'serum creatinine', 'AST', 'ALT', 'ggt', 'dental caries', 'tartar',
       'smoking']
df.columns

Index(['ID', 'gender', 'age', 'height(cm)', 'weight(kg)', 'waist(cm)',
       'eyesight(left)', 'eyesight(right)', 'hearing(left)', 'hearing(right)',
       'systolic', 'relaxation', 'fasting blood sugar', 'cholesterol',
       'triglyceride', 'HDL', 'LDL', 'hemoglobin', 'urine protein',
       'serum creatinine', 'AST', 'ALT', 'ggt', 'dental caries', 'tartar',
       'smoking'],
      dtype='object')

Adding new columns based on feedback

In [13]:
df['bmi'] = df['weight(kg)'] / (df['height(cm)'] ** 2)
df['upcratio'] = df['urine protein'] / df['serum creatinine']
df['LDLratio'] = df['LDL'] / df['cholesterol']
df['HDLratio'] = df['HDL'] / df['cholesterol']

Final look before saving cleaned data to csv

In [14]:
df.head()

,ID,gender,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),...,AST,ALT,ggt,dental caries,tartar,smoking,bmi,upcratio,LDLratio,HDLratio
0,0,0,40,155,60,81.3,1.2,1.0,1,1,...,18,19,27,0,1,0,0.002497,1.428571,0.586047,0.339535
1,1,0,40,160,60,81.0,0.8,0.6,1,1,...,22,19,18,0,1,0,0.002344,1.666667,0.661458,0.218750
2,2,1,55,170,60,80.0,0.8,0.8,1,1,...,21,16,22,0,0,1,0.002076,1.000000,0.623967,0.227273
3,3,1,40,165,70,88.0,1.5,1.5,1,1,...,19,26,18,0,1,0,0.002571,1.000000,0.701863,0.139752
4,4,0,40,155,60,86.0,1.0,1.0,1,1,...,16,14,22,0,0,0,0.002497,1.666667,0.581522,0.336957


In [15]:
#writing cleaned dataframe to csv
df.to_csv(data_path + 'cleaned_data.csv', encoding='utf-8', index=False)